In [36]:
import pandas as pd
import numpy as np
import math
import os

In [37]:
def haversine_distance(lat1, lon1, lat2, lon2, r=6371):
    coordinates = lat1, lon1, lat2, lon2
    phi1, lambda1, phi2 , lambda2 = [
        math.radians(c) for c in coordinates
    ]  
    a = (np.square(math.sin((phi2-phi1)/2)) + math.cos(phi1) * math.cos(phi2) * 
         np.square(math.sin((lambda2-lambda1)/2)))
    d = 2*r*math.asin(np.sqrt(a))
    return d
    
def add_cords(df, dfcor, dftarget, column, comp, lat, lng):
  for x in df[comp]:
    loc = dftarget.loc[dftarget[column].eq(x)]
    dfcor[lat].append(float(loc.lat))
    dfcor[lng].append(float(loc.lng))


codes = pd.read_csv('Codigos.csv')

capitals = pd.read_csv('Capitales.csv')
 
routes = pd.read_csv('Rutas.csv')

capitals = capitals[['capital','lat','lng']]
capitals['capital'] = capitals['capital'].str.upper()

routes = routes[['Source airport', 'Destination airport']]

codes = codes[(codes['Continent'] == "Eastern Europe") | (codes['Continent'] == "Europe")]
codes['City'] = codes['City'].str.upper()

codes = codes[((codes['City']).isin(capitals.capital))]
codes = codes.drop_duplicates(subset=["Code"],keep= "first")

flights = routes[(((routes['Source airport']).isin(codes.Code)) & ((routes['Destination airport']).isin(codes.Code)))]

In [38]:
cords = {'lat': [],'lng': []}
flight_cords = {'lat_st': [],'lng_st': [],'lat_end': [],'lng_end': []}

add_cords(codes, cords,capitals,"capital", "City","lat","lng")
cordsdf = pd.DataFrame(cords)

codes['lat'] = cordsdf['lat'].values
codes['lng'] = cordsdf['lng'].values

add_cords(flights, flight_cords, codes, "Code", "Source airport","lat_st","lng_st")
add_cords(flights, flight_cords, codes, "Code","Destination airport","lat_end","lng_end")
cordsdf = pd.DataFrame(flight_cords)


flights['lat_st'] = cordsdf['lat_st'].values
flights['lng_st'] = cordsdf['lng_st'].values
flights['lat_end'] = cordsdf['lat_end'].values
flights['lng_end'] = cordsdf['lng_end'].values


flights['distance'] = flights.apply(lambda row : haversine_distance(row['lat_st'],
                     row['lng_st'], row['lat_end'], row['lng_end']), axis = 1)


vec = {'start': [],'end': []}


for x in flights["Source airport"]:
    loc = codes.loc[codes["Code"].eq(x)]
    city = loc.City.values[0]
    vec["start"].append(str(city))


for x in flights["Destination airport"]:
    loc = codes.loc[codes["Code"].eq(x)]
    city = loc.City.values[0]
    vec["end"].append(str(city))

vecdf = pd.DataFrame(vec)
flights['Origin'] = vecdf['start'].values
flights['Destination'] = vecdf['end'].values


/var/folders/4x/by41vdp921s318y3mybq7fb80000gn/T/ipykernel_36704/1571547663.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights['lat_st'] = cordsdf['lat_st'].values
/var/folders/4x/by41vdp921s318y3mybq7fb80000gn/T/ipykernel_36704/1571547663.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights['lng_st'] = cordsdf['lng_st'].values
/var/folders/4x/by41vdp921s318y3mybq7fb80000gn/T/ipykernel_36704/1571547663.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [39]:
capitals

,capital,lat,lng
0,ANDORRA,42.5000,1.5165
1,TIRANA,41.3275,19.8189
2,VIENNA,48.2000,16.3666
3,SARAJEVO,43.8500,18.3830
4,BRUSSELS,50.8333,4.3333
5,SOFIA,42.6833,23.3167
6,MINSK,53.9000,27.5666
7,BERN,46.9167,7.4670
8,NICOSIA,35.1667,33.3666
9,PRAGUE,50.0833,14.4660


In [40]:
codes

,City,Country,Code,Continent,lat,lng
392,SARAJEVO,Bosnia and Herzegovina,SJJ,Eastern Europe,43.8500,18.3830
393,SOFIA,Bulgaria,SOF,Eastern Europe,42.6833,23.3167
397,BUDAPEST,Hungary,BUD,Eastern Europe,47.5000,19.0833
398,SKOPJE,Macedonia,SKP,Eastern Europe,42.0000,21.4335
399,BUCHAREST,Romania,BUH,Eastern Europe,44.4334,26.0999
400,BUCHAREST,Romania,OTP,Eastern Europe,44.4334,26.0999
403,MINSK,Belarus,MSQ,Eastern Europe,53.9000,27.5666
419,ZAGREB,Croatia (Hrvatska),ZAG,Eastern Europe,45.8000,16.0000
423,RIGA,Latvia,RIX,Eastern Europe,56.9500,24.1000
424,VILNIUS,Lithuania,VNO,Eastern Europe,54.6834,25.3166


In [41]:
flights=flights[['Source airport','Origin','Destination airport','Destination',"lat_st","lng_st","lat_end","lng_end","distance"]]
flights

,Source airport,Origin,Destination airport,Destination,lat_st,lng_st,lat_end,lng_end,distance
139,ARN,STOCKHOLM,TRF,OSLO,59.3508,18.0973,59.9167,10.7500,417.547216
164,TRF,OSLO,ARN,STOCKHOLM,59.9167,10.7500,59.3508,18.0973,417.547216
1406,FCO,ROME,TXL,BERLIN,41.8960,12.4833,52.5218,13.4015,1183.542757
1503,LHR,LONDON,TXL,BERLIN,51.5072,-0.1275,52.5218,13.4015,931.322599
1571,SJJ,SARAJEVO,TXL,BERLIN,43.8500,18.3830,52.5218,13.4015,1031.939115
...,...,...,...,...,...,...,...,...,...
66511,LGW,LONDON,TFS,SOFIA,51.5072,-0.1275,42.6833,23.3167,2015.140475
66523,LTN,LONDON,FCO,ROME,51.5072,-0.1275,41.8960,12.4833,1433.689911
66530,LTN,LONDON,TFS,SOFIA,51.5072,-0.1275,42.6833,23.3167,2015.140475
66585,TFS,SOFIA,LGW,LONDON,42.6833,23.3167,51.5072,-0.1275,2015.140475


In [42]:
file_name = 'data.csv'

if not os.path.exists(file_name):
    flights.to_csv(file_name, index=False)
else:
    print(f'El archivo {file_name} ya existe.')